# Scikit Learn & Kaggle
This week, we're going to learn about sci-kit learn.  For this lesson, I also want to introduce you to [Kaggle](https://www.kaggle.com/competitions) and its massive community of data scientists and learning materials.  Kaggle is a great place to get your feet wet with machine learning projects, and compare your approach to how others -- often hundreds of others -- have tackled the same problem.  

For this exercise, we're going to walk through a Kaggle competition that is perfect for learning about scikit learn: The [Titanic Challenge](https://www.kaggle.com/c/titanic). 

## Lesson Plan

1. The first step is to go to the site and learn a bit about the challenge: https://www.kaggle.com/c/titanic
2. Once you've got a sense of what we're trying to do (predict whether a person will survive), we'll walk through the basics of loading our packages and data.  
3. Then we'll spend a fair amount of time just looking at the data and gettting it into good shape for sklearn. 
4. Then we'll actually use sklearn and learn the basics.  You'll see pretty quickly why people use scikit learn -- it lets you try out many combinations of paremeters in many types of models with a simple set of commands. 
5. Then, we'll connect all our steps togehter into what sklearn calls a "pipeline". This will add a layer of complexity to our work, but will allow us to use something called "grid search" to train our model across the entire grid of possilbe features combintions and features weights for any classifier. 
5. Finally, once we have our pipeline up and running, we can submit it to Kaggle & officially join the data science community there. Don't worry, there is pretty much zero chance that we will win the competition.  But we will win a place in the heart of data science! 

# Import Packages and Load Data

In [ ]:
import numpy as np
import pandas as pd
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import preprocessing 

import statsmodels.api as sm 
import statsmodels.formula.api as smf 
from statsmodels.discrete.discrete_model import Logit

import matplotlib.pyplot as plt
% matplotlib  inline
import seaborn as sns; sns.set(color_codes=True)


## A True Test
Before we import out data, one important note: our test data set has no values for "Survived".  Yup, that rights.  And that's how it should be. We're going to build a model using our training data, then use that model to generate data for our test data.  This is a great real-world check.  We don't get to peak at the test data; we have to try to guess it from what know of the training data.  Bye bye traditional stats; hello machine learning & data science!  


In [ ]:
df_train = pd.read_csv('train.csv', header = 0, index_col = 'PassengerId')
df_test = pd.read_csv('test.csv', header = 0, index_col = 'PassengerId')

# I'm going to combine the two data sets for the purpose of data cleaning and feature generation.  
# But I'm going to keep track of them by labelling them "train" and "test" in the multi-index key. 
# That way I can select them later by using 'df.loc['train]' and df.loc['test'].  
# Multi-indexing is one of the ost powerful featurs of Pandas, and this is a perfect use for it. 

df = pd.concat([df_train, df_test], keys=["train", "test"])
df.columns

## 01 - grok the multi-index
Now let's take a quick look at the data. just type "df" and see if you can see how the data is now organized. 

In [ ]:
df

# Feature Generation

OK, some of these fields look a little opaque.  What does SibSp mean?  To help with that, here's a data dictionary: 

```
Data Dictionary
Survived: Survival	0 = No, 1 = Yes
Pclass: Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
Sex: Sex	
Age: Age in years
SibSp: # of siblings / spouses aboard the Titanic
Parch: # of parents / children aboard the Titanic
Ticket: Ticket number
Fare: Passenger fare
Cabin: Cabin number
Embarked: Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

Variable Notes

pclass: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...
Sibling = brother, sister, stepbrother, stepsister
Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.

```

In [ ]:
df.describe()

## Cleaning up the data

First, notice that Age and Fare are missing some values.  Let's fill those in.  There are much fancier ways to do this, but I'll just put in the median.  

In [ ]:
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Fare'] = df['Fare'].fillna(df['Fare'].median())

And what about Name, Sex, Ticket, Cabin, and Embarked?  Those are text fields.  As such, they can't be managed numerically.  Which means we have to transform them into numbers if we want to use them. Let's see if we can clean these up a bit. 

- "Name", for example is just text.  We might try to transform that using some fancy algorithms, but let's leave that alone for now.  
- "Sex" is a text field, too.  Let's turn that from a string into a one or a zero. 
- "Embarked" is also a text field, that should be turned into a set of dummy variables. 
- "Pclass" is a number, but it really should be a categorical variable fore each class. 

Let's do each right now. 

## 01 - pd.dummies

Using pandas excellent 'get_dummies' method, we can transform each of these variables (Sex, Embarked, and Pclass) into dummy binary variables. The command is really simple. 

```pd.get_dummies(train, columns = ['Pclass'])```

look up the documentation for that command, then see if you can return a dataframe that turns those three variables into dummies, while leaving the others intact. 

Easy, right? Let's do that for "Pclass", "Sex", and "Embarked" 

In [ ]:
df = pd.get_dummies(data=df, columns=['Pclass','Sex','Embarked'], drop_first=True).copy()
df.columns

Notice that I added 'drop_first=True' as a parameter to the get_dummies command.  Why do you think I do that? 

### A note on feature generation

The difference between a good model and a great model often depends on feature generation. Even when we just have a few variables like this, we can often generate hundreds of features from them by looking at various combinations.  For example, if we were really careful, we could probably figure out which passangers belong to which family and create a family ID.  Maybe members of smaller families do better than larger families, all other things being equal. Maybe single folks are more likely to survive because they are not held back by trying to save other family members.  Maybe members of families are more likely to survive because they are more motivated and can work together.  

We're not going to spend time adding all these features, but if this were something like detecting malignant tumors or scheduling police -- or perhaps even more importantly, finding rats! -- we'd be generating as many features as we could plausibly come up with.  As you'll see in a bit, having too many features is not as serious a problem for sklearn as it is for traditional frequentist linear models like traditional regression. 

All that is to say that feature generation is very important.  Try your hand at generating some features below. Start with an easy one: family size. Or try more if you like! 

In [ ]:
# eg, df['FamilySize'] = df['SibSp'] + df['Parch'] + 1


## 02 - Descriptive Statistics

Now use ```train.describe()``` to examine your data again. 

In [ ]:
df.loc['train'].describe().T

And now use the ```.corr()``` method to get basic correlations with Survived. 

In [ ]:
df.corr()['Survived'].sort_values()

In [ ]:
df.loc['train'].describe()

Right away, we can see a few things.  The people who were more likely to die were men, 3rd class passenger, and passengers who boarded at Southampton.  Conversely, women, first class passengers, and passengers who boarded at Cherbourgh were more likely to survive.  Also, we an see that fare price is positively correlated with survival.  

If we were in the old world, just using statsmodels, we'd make a formula, maybe like this: 

```Survived ~ Sex_female * Parch + Pclass_1 + Fare + Embarked_C + SibSp```

And then we'd feed that model into a logistic regression. Then we'd develop our various intuitions by playing around with the model on our training set, adding, subtracting, and interacting variables in the linear equation. Then we'd make some informed hypotheses about the relationships in the data and test them on a validation data set. And finally, when had our sense that we really knew what was going on, we test our best models on the test set. 

But we're entering the world of data science and machine learning. We're going to become more and more rigorous about automating all of that, and far more agnostic about our feaures and models.  We'll work hard to develop good features for our model, but once we do that, we want to try to take the human out of the loop of manipulating and comparing models.  We'll turn running through all those features and models over to sklearn, and just give it a way to evaluate and compare them. 

# sklearn

The first thing we need to do is turn our training data into two matrices of equal length, one called "X" with our features, and the other called "Y" with our dependent variable (oddly, in the ML world, the y variable is often called your "labels"). 

## preparing our data

First, we'll make a data set without columns we're not using at the moment. (Please remember that you probably *can* and *should* use them when doing thorough feature generation.we're just not doing now for the sake of simplicity.)



In [ ]:
df_tv = df.drop(['Cabin', 'Name', 'Ticket'], axis=1).copy().dropna()
tv = df_tv.loc['train']
X = tv.drop('Survived',  axis=1).as_matrix()
y = tv['Survived'].as_matrix()

sklearn doesn't read pandas dataframes -- instead it takes numpy matrices.  Let's compare what they look like: 

In [ ]:
X

In [ ]:
y

Wow! Just lists of numbers in brackets.  Yup.  No labels.  That's why we wait until the very end to switch over to numpy matrices.  Pandas is way, way friendlier. 

## Create a validation set. 

This may be a bit confusing, but validation is really just another word for a pre-test test. And since sklearn really only has labels for train and test, that's what we'll use here. Just think of it as our validation set. 

To split our data, I'm going to use the sklearn method designed for this and helpfully named 'train_test_split'. I'm going to split these 50-50, though as data sets get larger, the splits can be more like 80-20 or 90-10. 

In [ ]:
X_train, X_validate, y_train, y_validate = sklearn.model_selection.train_test_split(X, y, test_size=0.5, random_state=42)

Now I have four matrices, X and y for training, and X and y for validating. 

## Feature Selection

My next goal is to figure out which features to use.  If we look at our feature set now, we can see we have 9 features. 

In [ ]:
X_train.shape

That's not a lot; what if we had 1,000?  or 100,000?   To address this, we'd use the SelectKBest command, which allows us to select the number of features we want. By default, "best" uses anova classification fit statistics, though it can be changed if we want; but that's not import right now -- it's a good default. 

Let's try select 5 features. 

In [ ]:
X_new = sklearn.feature_selection.SelectKBest(k=5).fit_transform(X_train, y_train)
X_new.shape

We'll automate more of this later, but holy-moly that was fast.  And easy.  Now I can have whatever number of features make sense for my particular use case.  Yay! 

## Training, predicting and evaluating. 

Now we can train our model on the test data.  We can use just about any classifier we want -- logistic regression, random forest, you name it. Let's start with what is one of the most common classifiers in machine learning: familiar old logistic regression. 

In [ ]:
clf = sklearn.linear_model.LogisticRegression()  # describe our model
train_fit = clf.fit(X_train, y_train)            # train our model
y_prediction = train_fit.predict(X_validate)         # use our model to predict y varialbe for out validation data
y_prediction   # take a look at our validation predictions. 

# The ROC -- evaluating our model

Now we can evaluate how we did. The most common statistic for evaluating classifcation is the ROC, or the "Receiver Operator Curve".  It tells us how often we get false positives relative to true postivies. 


In [ ]:
from sklearn.metrics import roc_curve, auc
plt.style.use('ggplot')

fpr, tpr, _ = roc_curve(y_validate, y_prediction)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange',
         lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
df_test = df.loc['test'].drop(['Cabin', 'Name', 'Ticket'], axis=1)
X_test = df_test.drop('Survived',  axis=1).as_matrix()
y_test = train_fit.predict(X_test)         # use our model to predict y varialbe for out validation data

Now we just save it to upload to Kaggle.  Kaggle expects integers and a header row, so we'll give them that. 

In [ ]:
df.loc['test','Survived'] = y_test
df.loc['test','Survived'].astype(int).to_csv('results.csv', header=True)

# That's it! 

We'll stop there for now.  

Go and upload your results to Kaggle. Then see if you can improve your score!  
